In [1]:
#  Python imports
#
#  Notes:
#  * Make sure you install the packages in requirements.txt
#  * Make sure you setup your KAGGLE secrets via env vars.
#
import os
import keras
import keras_nlp
from keras_nlp.models import GemmaBackbone, BertBackbone
from keras.models import load_model
import kagglehub
from langchain.schema.runnable import Runnable
from typing import Any, Optional
import tensorflow as tf
from keras.config import disable_interactive_logging
import gc

In [2]:
# Useful functions
#
def makebold(txt):
    return '\x1b[1;30m'+txt+'\x1b[0m'
def makeblue(txt):
    return '\x1b[1;34m'+txt+'\x1b[0m'
def makegreen(txt):
    return '\x1b[1;32m'+txt+'\x1b[0m'
def makered(txt):
    return '\x1b[1;33m'+txt+'\x1b[0m'

In [6]:
#
# Test a basic prompt completion a minimal Langchain Runnable
# integrating previous cherry-picked history in a loop.
#
class LMRunnable(Runnable):
    def __init__(self, model: keras_nlp.models.GemmaCausalLM):
        self.model = model
    def invoke(self, input: Any, config: Optional[dict] = None) -> str:
        prompt = str(input)
        output = self.model.generate(prompt, max_length=2048)
        return output

# Use the Gemma model as a Langchain Runnable
model_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_instruct_2b_en")
sampler = keras_nlp.samplers.TopKSampler(k=3, seed=2)
model_lm.compile(sampler=sampler)
runnable = LMRunnable(model_lm)

system_message = "You are a writer of fiction."
first_time = True
for i in range(1,5):
    
    # Create the next prompt via manual formatting of parts
    if first_time:
        initial_prompt = "Tell me a story. My name is Daniel."
        runnable_prompt = f"{system_message}\n\n{initial_prompt}"
        first_time = False
    else:
        cherry_picked_history = " ".join( runnable_prompt_completion.strip().split("\n\n")[-2:] )
        runnable_prompt = f"{system_message}. Use the following context and continue telling the story.\n\n{cherry_picked_history}"
    print(makebold("Runnable prompt %d:\n" % i) +  makegreen("\"" + runnable_prompt + "\"") )

    # Complete the prompt via Runnable invoke()
    runnable_prompt_completion = runnable.invoke(runnable_prompt)
    print(makebold("\nRunnable completion %d:\n" % i) + makeblue("\"" + runnable_prompt_completion + "\""))

runnable = None # release model handle
model_lm = None # try to clean up model memory
gc.collect() # Run python memory collector

Runnable prompt 1:
"You are a writer of fiction.

Tell me a story. My name is Daniel."

Runnable completion 1:
"You are a writer of fiction.

Tell me a story. My name is Daniel.

I woke up on the beach, the salty air filling my lungs. The sun was shining brightly, casting long shadows on the sand and the sea. I stretched my arms out to the ocean, and the waves lapped at my ankles, sending shivers down my spine.

I had been dreaming about this trip for weeks. I had heard about a hidden cove on the coast, a place that was untouched and pristine. I had finally found it, and I couldn't wait to explore it.

I took a deep breath and stepped out onto the beach. The sand was warm and soft under my feet, and the sun was warm on my skin. I smiled, and I knew that I was in a special place.

I wandered along the shore, looking for the perfect spot to swim. I found a secluded cove that was hidden from the crowds. The water was crystal clear, and the sand was pristine. I sat down on a rock and took 

90306